In [ ]:
fileNameGood = 'good_'
fileNameBad = 'bad_'

# For action - Good Postures
# Loop through videos
for sequence in range(1,31):
    fileName = fileNameGood+str(sequence)+'.mp4'
    cap = cv2.VideoCapture(fileName)

    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
        # Loop through video length aka sequence length
        for frame_num in range(sequence_length):

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)
                
            # NEW Apply wait logic
            if frame_num == 0: 
                cv2.putText(image, 'STARTING COLLECTION FOR GOOD POSTURES', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for Good Postures Video Number {}'.format(sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.waitKey(2000)
            else: 
                cv2.putText(image, 'Collecting frames for Good Postures Video Number {}'.format(sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
            # Show to screen
            cv2.imshow('Real Time Posture Correction System (RTPCS) - Good Postures', image)
                
            # NEW Export keypoints
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
            np.save(npy_path, keypoints)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                
        cap.release()
        cv2.destroyAllWindows()

# For action - Bad Postures
# Loop through videos
for sequence in range(1,31):
    fileName = fileNameBad+str(sequence)+'.mp4'
    cap = cv2.VideoCapture(fileName)

    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
        # Loop through video length aka sequence length
        for frame_num in range(sequence_length):

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)
                
            # NEW Apply wait logic
            if frame_num == 0: 
                cv2.putText(image, 'STARTING COLLECTION FOR BAD POSTURES', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for Bad Postures Video Number {}'.format(sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.waitKey(2000)
            else: 
                cv2.putText(image, 'Collecting frames for Bad Postures Video Number {}'.format(sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
            # Show to screen
            cv2.imshow('Real Time Posture Correction System (RTPCS) - Bad Postures', image)
                
            # NEW Export keypoints
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
            np.save(npy_path, keypoints)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
# Instantiating a sequential model
model = Sequential()

# 3 LSTM layers in total with 1st LSTM layer with 64 units and input shape with 100 frames and 258 keypoint values
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(100,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

# 3 Fully connected Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

# Less Data Required for such model, faster to train, and faster detections in real-time